In [1]:
import pandas as pd
import numpy as np

In [2]:
df2010 = pd.read_csv('../Data/s2010.tdf', sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
df2011 = pd.read_csv( '../Data/s2011.tdf',sep='\t', engine='python', names=['symb', 'retdate', 'opn', 'high', 'low','cls', 'vol', 'exch'], parse_dates = ['retdate'])
dffnd = pd.read_csv( '../Data/fnd.tdf',sep='\t', engine='python', names=['gvkey', 'datadate', 'fyear', 'indfmr','consol', 'popsrc', 'datafmt' , 'tic', 'cusip', 'conm', 'fyr', 'cash', 'dp','ebitda', 'emp', 'invt', 'netinc', 'ppent', 'rev' , 'ui', 'cik'])
dfMTA = pd.read_csv('../raw_data/mta/MTA_Hourly.tdf', sep='\t',engine='python', names=['plaza', 'mtadt', 'hr', 'direction', 'vehiclesez','vehiclescash'], parse_dates = ['mtadt'])
dfTrans = pd.read_csv('../raw_data/soapData.tdf', sep='\t',engine='python', names = ['orderid', 'userid', 'trans', 'type', 'local','trans_dt', 'units', 'coupon', 'months', 'amt' ])

In [3]:
df2010.dtypes

symb               object
retdate    datetime64[ns]
opn               float64
high              float64
low               float64
cls               float64
vol                 int64
exch               object
dtype: object

# First Five  

1. 

In [6]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
left.columns = ['symb', 'max_2010']
right = df2011.groupby(df2011.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
right.columns = ['symb', 'max_2011']
pd.merge(left, right, on = ['symb'], how = 'inner')

,symb,max_2010,max_2011
0,A,29.9857,37.6109
1,AA,52.3500,54.3900
2,AAME,2.0500,2.2200
3,AAN,23.8000,29.1600
4,AAON,8.6697,10.5955
...,...,...,...
3330,ZIXI,4.4900,4.7500
3331,ZN,7.2000,6.9800
3332,ZNH,38.6500,35.4000
3333,ZTR,15.8400,14.5600


2. 

In [12]:
right = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['max']})
right.columns = ['symb', 'last2010']
left = df2011.groupby(df2011.loc[:, 'symb'], as_index = False).agg({'retdate': ['max']})
left.columns = ['symb', 'last2011']
pd.merge(left, right, on = ['symb'], how = 'left')

,symb,last2011,last2010
0,A,2011-12-30,2010-12-31
1,AA,2011-12-30,2010-12-31
2,AAME,2011-12-30,2010-12-31
3,AAN,2011-12-30,2010-12-31
4,AAON,2011-12-30,2010-12-31
...,...,...,...
3529,ZNGA,2011-12-30,NaT
3530,ZNH,2011-12-30,2010-12-31
3531,ZTR,2011-12-30,2010-12-31
3532,ZUMZ,2011-12-30,2010-12-31


3.

In [15]:
left = df2010.loc[:, ['symb']].drop_duplicates()
right = df2010.loc[:, ['retdate']].drop_duplicates()
pd.merge(left, right, how = 'cross')

,symb,retdate
0,AAME,2010-04-07
1,AAME,2010-04-13
2,AAME,2010-06-11
3,AAME,2010-11-23
4,AAME,2010-02-01
...,...,...
847219,EDF,2010-11-12
847220,EDF,2010-02-24
847221,EDF,2010-08-13
847222,EDF,2010-06-08


4. 

In [20]:
left = df2010.loc[:, ['symb']].drop_duplicates()
right = df2010.loc[:, ['retdate']].drop_duplicates()
pd.merge(left, right, how = 'cross').shape[0]

847224

In [19]:
df2010.shape[0]

816066

5. 

In [26]:
left = df2010.groupby(df2010.loc[:, 'retdate'].dt.strftime('%B')).agg({'retdate': ['count']}).reset_index()
left.columns = ['Month', 'nrows2010']
right = df2011.groupby(df2011.loc[:, 'retdate'].dt.strftime('%B')).agg({'retdate': ['count']}).reset_index()
right.columns = ['Month', 'nrows2011']
pd.merge(left, right, on = ['Month'], how = 'inner')

,Month,nrows2010,nrows2011
0,April,67243,67995
1,August,71731,79524
2,December,73041,73458
3,February,60116,63859
4,January,60003,66714
5,July,68232,68855
6,June,71170,75446
7,March,73087,77618
8,May,64394,71783
9,November,69398,73211


# Main Problems

1. 

In [6]:
left = df2010.loc[:, ['symb']].drop_duplicates()
left.columns = ['symb_2010']
right = df2011.loc[:, ['symb']].drop_duplicates()
right.columns = ['symb_2011']
temp = pd.merge(left, right, left_on = 'symb_2010', right_on = 'symb_2011', how = 'left', indicator = True)
temp.loc[(temp.loc[:, '_merge'] == 'left_only'), 'symb_2010'].count()

27

2.

In [9]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['min']})
left.columns = ['symb', 'firstday']
right = df2010.loc[:, ['symb', 'retdate', 'cls']]
pd.merge(left, right, left_on = ['symb', 'firstday'], right_on = ['symb', 'retdate'], how = 'left')

,symb,firstday,retdate,cls
0,A,2010-01-04,2010-01-04,22.3891
1,AA,2010-01-04,2010-01-04,49.9500
2,AAME,2010-01-04,2010-01-04,1.3200
3,AAN,2010-01-04,2010-01-04,18.5400
4,AAON,2010-01-04,2010-01-04,5.9230
...,...,...,...,...
3357,ZIXI,2010-01-04,2010-01-04,1.7800
3358,ZN,2010-01-04,2010-01-04,7.2000
3359,ZNH,2010-01-04,2010-01-04,15.9800
3360,ZTR,2010-01-04,2010-01-04,15.6800


3. 

In [13]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['min']})
left.columns = ['symb', 'firstday']
right = df2010.loc[:, ['symb', 'retdate', 'cls']]
new_left = pd.merge(left, right, left_on = ['symb', 'firstday'], right_on = ['symb', 'retdate'], how = 'left').loc[:, ['symb', 'cls']].rename(columns = {'cls': 'firstdaycls'})
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['max']})
left.columns = ['symb', 'lastday']
right = df2010.loc[:, ['symb', 'retdate', 'cls']]
new_right = pd.merge(left, right, left_on = ['symb', 'lastday'], right_on = ['symb', 'retdate'], how = 'left').loc[:, ['symb', 'cls']].rename(columns = {'cls': 'lastdaycls'})
pd.merge(new_left, new_right, how = 'inner', on = 'symb')

,symb,firstdaycls,lastdaycls
0,A,22.3891,29.6352
1,AA,49.9500,46.1700
2,AAME,1.3200,2.0300
3,AAN,18.5400,20.3900
4,AAON,5.9230,8.3585
...,...,...,...
3357,ZIXI,1.7800,4.2700
3358,ZN,7.2000,4.7800
3359,ZNH,15.9800,30.5100
3360,ZTR,15.6800,14.2400


4. 

In [10]:
left = df2010.loc[(df2010.loc[:, 'retdate'].dt.strftime('%A') == 'Monday'), ['symb', 'retdate', 'cls']]
right = df2010.loc[(df2010.loc[:, 'retdate'].dt.strftime('%A') == 'Tuesday'), ['symb', 'retdate', 'cls']].rename(columns={'cls': 'cls_nextday'})
right.loc[:, 'daybefore'] = right.loc[:, 'retdate'] - pd.Timedelta(days = 1)
right
pd.merge(left, right.drop(columns = ['retdate']), left_on = ['symb', 'retdate'], right_on = ['symb', 'daybefore'], how = 'inner').drop(columns = ['daybefore'])

,symb,retdate,cls,cls_nextday
0,AAME,2010-02-01,1.3500,1.3900
1,AAON,2010-02-01,6.0889,6.2015
2,AAPL,2010-02-01,27.8186,27.9800
3,AAWW,2010-02-01,37.3700,38.3400
4,AAXJ,2010-02-01,53.0700,53.4200
...,...,...,...,...
152242,YUM,2010-01-25,34.2800,34.3300
152243,ZB-A,2010-01-25,16.2500,16.3500
152244,ZF,2010-01-25,12.8800,12.8800
152245,ZNH,2010-01-25,16.9400,16.4600


5. 

In [33]:
left = df2010.loc[:, ['symb']].drop_duplicates()
right = dffnd.loc[:, ['tic']].drop_duplicates()
temp = pd.merge(left, right, left_on='symb', right_on = 'tic', how = 'outer', indicator = True)
nboth = temp.loc[(temp.loc[:, '_merge'] == 'both'), '_merge'].count()
n2010 = temp.loc[(temp.loc[:, '_merge'] == 'left_only'), '_merge'].count()
nfnd = temp.loc[(temp.loc[:, '_merge'] == 'right_only'), '_merge'].count()
pd.DataFrame({'both': nboth, '2010': n2010, 'fnd': nfnd}, index = [1])

,both,2010,fnd
1,3119,243,10060


6. 

In [28]:
temp = pd.merge(left = df2010.loc[:, ['symb']].drop_duplicates(), right = dffnd.loc[:, ['tic']].drop_duplicates(), how = 'outer', left_on = ['symb'], right_on = ['tic'], indicator= True)
temp.groupby(temp.loc[:, '_merge']).agg({'_merge': ['count']}).reset_index()

_merge       
               count
0   left_only    243
1  right_only  10060
2        both   3119

7. 

In [39]:
left = df2010.groupby(df2010.loc[:, 'retdate'].dt.strftime('%A')).agg({'cls': ['mean']}).reset_index()
left.columns = ['dow', 'cls_2010']
right = df2011.groupby(df2011.loc[:, 'retdate'].dt.strftime('%A')).agg({'cls': ['mean']}).reset_index()
right.columns = ['dow', 'cls_2011']
pd.merge(left, right, on = 'dow', how = 'inner')

,dow,cls_2010,cls_2011
0,Friday,2684.083145,1876.194780
1,Monday,2732.947968,1885.520858
2,Thursday,2698.283494,1908.122748
3,Tuesday,2704.411781,1895.687620
4,Wednesday,2699.352646,1893.319519


In [54]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
left.columns = ['symb', 'max_cls']
left.loc[(left.loc[:, 'max_cls'] < 50), 'type'] = 'below50'
left.loc[(left.loc[:, 'max_cls'] > 50) & (left.loc[:, 'max_cls'] < 100), 'type'] = 'between'
left.loc[(left.loc[:, 'max_cls'] > 100), 'type'] = 'above100'
right = dffnd.loc[(dffnd.loc[:, 'fyear'] == 2010), :].groupby(dffnd.loc[:, 'tic']).agg({'netinc': ['max']}).reset_index()
right.columns = ['symb', 'netinc']
right

,symb,netinc
0,0015B,30.998
1,0030B,22.828
2,0032A,-3530.000
3,0033A,-2812.000
4,0038A,204.982
...,...,...
11068,ZUT.,NaN
11069,ZX,49.662
11070,ZYCI,NaN
11071,ZYXI,0.350


8. 

In [57]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'cls': ['max']})
left.columns = ['symb', 'max_cls']
left.loc[(left.loc[:, 'max_cls'] < 50), 'type'] = 'below50'
left.loc[(left.loc[:, 'max_cls'] > 50) & (left.loc[:, 'max_cls'] < 100), 'type'] = 'between'
left.loc[(left.loc[:, 'max_cls'] > 100), 'type'] = 'above100'
right = dffnd.loc[(dffnd.loc[:, 'fyear'] == 2010), :].groupby(dffnd.loc[:, 'tic']).agg({'netinc': ['max']}).reset_index()
right.columns = ['symb', 'netinc']
right = right.loc[~(right.loc[:, 'netinc'].isna()), :]
out = pd.merge(left, right, on = 'symb', how = 'inner')
out.groupby(out.loc[:, 'type'], as_index = False).agg({'netinc': ['mean']})

,type,netinc
,,mean
0,above100,963.670744
1,below50,325.222616
2,between,1482.581835


# Extra Problems

1. Create a dataset which contains the first day that each symbol is traded in 2010, the last day that the symbol is traded in 2011 and only includes those symbols which are in both 2010 and 2011

In [4]:
left = df2010.groupby(df2010.loc[:, 'symb'], as_index=False).agg({'retdate': ['min']})
left.columns = ['symb', 'firstday']
right = df2011.groupby(df2011.loc[:, 'symb'], as_index=False).agg({'retdate': ['max']})
right.columns = ['symb', 'lastday']
pd.merge(left, right, on = 'symb', how = 'inner')

,symb,firstday,lastday
0,A,2010-01-04,2011-12-30
1,AA,2010-01-04,2011-12-30
2,AAME,2010-01-04,2011-12-30
3,AAN,2010-01-04,2011-12-30
4,AAON,2010-01-04,2011-12-30
...,...,...,...
3330,ZIXI,2010-01-04,2011-12-30
3331,ZN,2010-01-04,2011-12-30
3332,ZNH,2010-01-04,2011-12-30
3333,ZTR,2010-01-04,2011-12-30


2. For those symbols which had a closing price larger than $100 anytime in 2010, return the symbol,first day that it was traded in 2010 and all the dates that it had a closing price larger than $200 in2010. If the symbol was never above $200, return no rows for it.

In [11]:
lst = df2010.loc[(df2010.loc[:, 'cls'] > 100), 'symb'].drop_duplicates()
firstday = df2010.loc[(df2010.loc[:, 'symb'].isin(lst)), ['retdate']].groupby(df2010.loc[:, 'symb']).agg({'retdate': ['min']}).reset_index()
firstday.columns = ['symb', 'retdate']
temp = df2010.loc[(df2010.loc[:, 'symb'].isin(lst)) & (df2010.loc[:, 'cls'] > 200), ['symb', 'retdate']]
pd.merge(firstday, temp, on = ['symb', 'retdate'], how = 'outer').sort_values(['symb', 'retdate'])

,symb,retdate
0,ABIO,2010-01-04
3584,ABIO,2010-03-26
1587,ABIO,2010-03-29
3563,ABIO,2010-03-30
3229,ABIO,2010-03-31
...,...,...
7653,Y,2010-12-28
7803,Y,2010-12-29
5405,Y,2010-12-30
5613,Y,2010-12-31


3. What are the first and last date listed for each symbol in 2010? Be careful to return this for each symb.

In [12]:
df2010.groupby(df2010.loc[:, 'symb'], as_index = False).agg({'retdate': ['min', 'max']})

symb    retdate           
                  min        max
0        A 2010-01-04 2010-12-31
1       AA 2010-01-04 2010-12-31
2     AAME 2010-01-04 2010-12-31
3      AAN 2010-01-04 2010-12-31
4     AAON 2010-01-04 2010-12-31
...    ...        ...        ...
3357  ZIXI 2010-01-04 2010-12-31
3358    ZN 2010-01-04 2010-12-31
3359   ZNH 2010-01-04 2010-12-31
3360   ZTR 2010-01-04 2010-12-31
3361  ZUMZ 2010-01-04 2010-12-31

[3362 rows x 3 columns]

4. For each symbol that appears anywhere in 2010, calculate the number of missing trading days thatit has in each month in 2010. This should return three columns: symbol, month, number of missing values.

In [24]:
lst = df2010.loc[:, ['symb']].drop_duplicates()
temp = df2010.groupby([df2010.loc[:, 'symb'], df2010.loc[:, 'retdate'].dt.month]).agg({'retdate': ['count']}).reset_index()
temp.columns = ['symb', 'month', 'ndays']
temp2 = temp.groupby(temp.loc[:, 'month'], as_index = False).agg({'ndays': 'max'})
temp2.columns = ['month', 'maxdays']
left = df2010.groupby([df2010.loc[:, 'symb'], df2010.loc[:, 'retdate'].dt.month]).agg({'retdate': ['count']}).reset_index()
left.columns = ['symb', 'month', 'ndays']
out = pd.merge(left, temp2, on = 'month', how = 'left')
out.assign(nmissing = out.loc[:, 'maxdays']-out.loc[:, 'ndays']).drop(columns = ['maxdays', 'ndays'])

,symb,month,nmissing
0,A,1,0
1,A,2,0
2,A,3,0
3,A,4,0
4,A,5,0
...,...,...,...
38975,ZUMZ,8,0
38976,ZUMZ,9,0
38977,ZUMZ,10,0
38978,ZUMZ,11,0


5. Create a dataset which is 10 rows by 3 columns. The first column should be the year, the secondcolumn should be the day-of-the-week and the third column should be the average closing price of all stocks for that day-of-the-week. Include both 2010 and 2011.

In [34]:
left = df2010.groupby(df2010.loc[:, 'retdate'].dt.day_of_week).agg({'cls': ['mean']}).reset_index()
left.columns = ['dow', 'avgcls']
left.loc[:, 'year'] = 2010
right = df2011.groupby(df2011.loc[:, 'retdate'].dt.day_of_week).agg({'cls': ['mean']}).reset_index()
right.columns = ['dow', 'avgcls']
right.loc[:, 'year'] = 2011
pd.concat([left, right], axis = 0)

,dow,avgcls,year
0,0,2732.947968,2010
1,1,2704.411781,2010
2,2,2699.352646,2010
3,3,2698.283494,2010
4,4,2684.083145,2010
0,0,1885.520858,2011
1,1,1895.687620,2011
2,2,1893.319519,2011
3,3,1908.122748,2011
4,4,1876.194780,2011


6. How many cars (total), on an average day, go through each toll plaza in both directions combined (return a row for each toll plaza)? Make sure to sum up to the day level before computing the average

In [42]:
temp = dfMTA.copy().assign(traffic = dfMTA.loc[:, 'vehiclescash'] + dfMTA.loc[:, 'vehiclesez']).groupby(['plaza', 'mtadt']).agg({'traffic': ['sum']}).reset_index()
temp.columns = ['plaza', 'mtadt', 'dailytraffic']
temp.groupby('plaza').agg({'dailytraffic': ['mean']})

,dailytraffic
,mean
plaza,
1,78620.134270
2,87264.610851
3,111212.727557
4,62292.127736
5,21207.228337
6,21238.190867
7,77821.620219
8,47096.385848


7. Which day-of-the-week (Monday, Tuesday, etc.) has the highest number of cars going through Plaza#1, both directions combined, with EZ pass? This should be the total number of cars over the entiretime period in the dataset

In [49]:
dfMTA.loc[(dfMTA.loc[:, 'plaza'] == 1), :].groupby(dfMTA.loc[:, 'mtadt'].dt.day_of_week).agg({'vehiclesez': ['sum']})

,vehiclesez
,sum
mtadt,
0,20892320
1,21722051
2,22057808
3,22568606
4,22848880
5,19050327
6,17925310


8. Which day-of-the-week-plaza combination has the lowest percentage of users cars using the EZ pass in the outbound direction? In other words, if you look at outbound cars through each plaza, whichday of the week has the lowest percentage of cars using EZ pass. You can compute the percentageover the entire time period

In [54]:
totez = dfMTA.loc[(dfMTA.loc[:, 'direction'] == 'O'), 'vehiclesez'].sum()
temp = dfMTA.loc[(dfMTA.loc[:, 'direction'] == 'O'), :].groupby(['plaza', dfMTA.loc[:, 'mtadt'].dt.day_of_week]).agg({'vehiclesez': ['sum']}).reset_index()
temp.columns = ['plaza', 'dow', 'traffic']
temp.loc[:, 'perc'] = temp.loc[:, 'traffic']/totez*100
temp.nsmallest(1, 'perc')

,plaza,dow,traffic,perc
41,6,6,2537693,0.405399


9. Calculate the average number of cars going through Plaza #1, outbound, with EZ pass for eachday-of-the-week. This should be a daily average and should return 7 rows

In [55]:
dfMTA.loc[(dfMTA.loc[:, 'direction'] == 'O') & 
          (dfMTA.loc[:, 'plaza'] == 1), :].groupby(dfMTA.loc[:, 'mtadt'].dt.day_of_week).agg({'vehiclesez': ['mean']})

,vehiclesez
,mean
mtadt,
0,1242.838570
1,1281.330260
2,1301.146918
3,1319.679303
4,1311.702072
5,1072.556994
6,1062.775956


10. In an average week on Plaza #1 with EZ pass (outbound), what percentage of cars go through eachday? (E.g. basically the above, but this time percent of total)

In [57]:
tot = dfMTA.loc[(dfMTA.loc[:, 'direction'] == 'O') & 
          (dfMTA.loc[:, 'plaza'] == 1), 'vehiclesez'].sum()
temp = dfMTA.loc[(dfMTA.loc[:, 'direction'] == 'O') & 
          (dfMTA.loc[:, 'plaza'] == 1), :].groupby(dfMTA.loc[:, 'mtadt'].dt.day_of_week).agg({'vehiclesez': ['sum']}).reset_index()
temp.columns = ['dow', 'traffic']
temp.loc[:, 'perc'] = temp.loc[:, 'traffic']/tot*100
temp

,dow,traffic,perc
0,0,10917094,14.466068
1,1,11255205,14.914094
2,2,11398047,15.103371
3,3,11592063,15.360459
4,4,11521991,15.267607
5,5,9447082,12.518178
6,6,9335424,12.370222


11. For each plaza, what was the change (percent) in average number of cars on a Monday using EZ-passin both directions, between 2015 and 2016? (Calculate the average number of cars for a Monday in 2015 and 2016 and then calculate the percentage change based off of that.

In [58]:
cars2015 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2015) & (dfMTA.loc[:, 'mtadt'].dt.day_of_week == 1), 'vehiclesez'].mean()
cars2016 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2016) & (dfMTA.loc[:, 'mtadt'].dt.day_of_week == 1), 'vehiclesez'].mean()
(cars2016-cars2015)/cars2015*100

4.8254504523234125

12. Calculate, for each hour, plaza and day-of-the-week (so 7 · 24 rows per plaza), the ratio of inbound to outbound traffic

In [62]:
inbound = dfMTA.loc[(dfMTA.loc[:, 'direction'] == 'I'), :].assign(traffic = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
    groupby(['hr', 'plaza', dfMTA.loc[:, 'mtadt'].dt.day_of_week]).agg({'traffic': ['sum']}).reset_index()
inbound.columns = ['hr', 'plaza', 'dow', 'in']
outbound = dfMTA.loc[(dfMTA.loc[:, 'direction'] == 'O'), :].assign(traffic = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
    groupby(['hr', 'plaza', dfMTA.loc[:, 'mtadt'].dt.day_of_week]).agg({'traffic': ['sum']}).reset_index()
outbound.columns = ['hr', 'plaza', 'dow', 'out']
temp = pd.merge(inbound, outbound, on = ['hr', 'plaza', 'dow'], how = 'inner')
temp.loc[:, 'ratio'] = temp.loc[:, 'in']/temp.loc[:, 'out']
temp

,hr,plaza,dow,in,out,ratio
0,0,1,0,282804,250637,1.128341
1,0,1,1,245628,180492,1.360880
2,0,1,2,251547,186395,1.349537
3,0,1,3,270229,198185,1.363519
4,0,1,4,308409,232580,1.326034
...,...,...,...,...,...,...
1507,23,9,2,343352,437534,0.784744
1508,23,9,3,401712,494301,0.812687
1509,23,9,4,469392,584582,0.802953
1510,23,9,5,665033,757467,0.877970


13. Using a join, create a dataset with three columns and 7 rows. The first column should be the DOW,the second column should be the average number of cars, per-day-of-the-week, through toll Plaza #1in either direction with an EZ pass in 2016 and the final column should be the average number of cars, per-day-of-the-week, through tool Plaza #2 with an EZ pass in 2015

In [64]:
cars2016 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2016) & 
                     (dfMTA.loc[:, 'plaza'] == 1), :].groupby(dfMTA.loc[:, 'mtadt'].dt.day_of_week).agg({'vehiclesez': ['mean']}).reset_index()
cars2016.columns = ['dow', 'cars2016']
cars2015 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2015) & 
                     (dfMTA.loc[:, 'plaza'] == 2), :].groupby(dfMTA.loc[:, 'mtadt'].dt.day_of_week).agg({'vehiclesez': ['mean']}).reset_index()
cars2015.columns = ['dow', 'cars2015']
pd.merge(cars2016, cars2015, on = 'dow', how = 'inner')

,dow,cars2016,cars2015
0,0,1327.718750,1608.459535
1,1,1391.796074,1660.452724
2,2,1425.364583,1737.283253
3,3,1450.856971,1777.120283
4,4,1438.693109,1769.419471
5,5,1237.137579,1403.962740
6,6,1181.461939,1417.513221


14. Create a dataset which contains twenty-four rows and two columns. The first column represents thehour and the second column represents the max number of EZ pass cars, during that hour, outbound,through Plaza #1.

In [65]:
dfMTA.loc[(dfMTA.loc[:, 'plaza'] == 1) &
          (dfMTA.loc[:, 'direction'] == 'O')].groupby(dfMTA.loc[:, 'hr'], as_index = False).agg({'vehiclesez': ['max']})


,vehiclesez
,max
0,1430
1,1021
2,990
3,736
4,842
5,1821
6,3066
7,3040
8,2711


15. Create a dataset which contains 24x7 rows and two columns. The first column represents the DOW-hour combination (you may need to combine two columns using “||” or the concatenate operator)and the second represents the max number of EZ pass cars, during that hour-day, through Plaza #1in the outbound direction

In [8]:
dfMTA.loc[(dfMTA.loc[:, 'plaza'] == 1) & 
          (dfMTA.loc[:, 'direction'] == 'O'), :].assign(dow_hr = dfMTA.loc[:, 'mtadt'].dt.day_of_week.astype('string') + "-" +  dfMTA.loc[:, 'hr'].astype('string')).\
            groupby('dow_hr').agg({'vehiclesez': ['max']}).reset_index()


,dow_hr,vehiclesez
,,max
0,0-0,1067
1,0-1,509
2,0-10,2452
3,0-11,2220
4,0-12,2153
...,...,...
163,6-5,701
164,6-6,765
165,6-7,875


16. Using at least one join, create a dataset which contains twenty-four rows and 4 columns. Each rowshould represent an hour, and the first column should be an hour identifier. Column #2 shouldcontain the maximum number of EZ pass cars, in the inbound direction, through Plaza #1 duringthat hour, Column #3 should contain the minimum number of outbound EZ-pass cars, during thathour, through Plaza #2 and Column #3 should be the maximum number of EZ-pass cars in eitherdirections combined, during that hour, on Plaza 3.

In [11]:
df1 = dfMTA.loc[(dfMTA.loc[:, 'plaza'] == 1) &
                (dfMTA.loc[:, 'direction'] == 'I'), :].groupby('hr', as_index = False).agg({'vehiclesez': ['max']})
df1.columns = ['hr', 'max1']
df2 = dfMTA.loc[(dfMTA.loc[:, 'plaza'] == 2) &
                (dfMTA.loc[:, 'direction'] == 'O'), :].groupby('hr', as_index = False).agg({'vehiclesez': ['min']})
df2.columns = ['hr', 'min2']
df3 = dfMTA.loc[(dfMTA.loc[:, 'plaza'] ==3), :].groupby('hr', as_index = False).agg({'vehiclesez': ['max']})
df3.columns = ['hr', 'max3']
df4 = pd.merge(df1, df2, on = 'hr', how = 'inner')
pd.merge(df4, df3, on = 'hr', how = 'inner')

,hr,max1,min2,max3
0,0,1860,0,2753
1,1,1274,0,1626
2,2,1085,0,1052
3,3,961,0,913
4,4,813,0,1223
5,5,1135,0,2718
6,6,2110,0,4116
7,7,2900,0,4572
8,8,2962,0,4452
9,9,2744,0,3925


17. Create a dataset which contains the following columns: hour, day-of-the-week, plaza, the ratio ofinbound to outbound traffic in 2014 and the ratio of inbound to outbound traffic in 2013.

In [31]:
in2014 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2014) &
                   (dfMTA.loc[:, 'direction'] == 'I'), :].\
                    assign(carsin = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
                        groupby(['hr', dfMTA.loc[:, 'mtadt'].dt.day_of_week, 'plaza']).agg({'carsin': ['sum']}).reset_index()
in2014.columns = ['hr', 'dow', 'plaza', 'carsin']
out2014 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2014) &
                   (dfMTA.loc[:, 'direction'] == 'O'), :].\
                    assign(carsout = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
                        groupby(['hr', dfMTA.loc[:, 'mtadt'].dt.day_of_week, 'plaza']).agg({'carsout': ['sum']}).reset_index()
out2014.columns = ['hr', 'dow', 'plaza', 'carsout']
left = pd.merge(in2014, out2014, on = ['hr', 'dow', 'plaza'], how = 'inner')
left.loc[:, 'ratio2014'] = right.loc[:, 'carsin']/right.loc[:, 'carsout']
left.drop(columns = ['carsin', 'carsout'], inplace=True)
in2013 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2013) &
                   (dfMTA.loc[:, 'direction'] == 'I'), :].\
                    assign(carsin = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
                        groupby(['hr', dfMTA.loc[:, 'mtadt'].dt.day_of_week, 'plaza']).agg({'carsin': ['sum']}).reset_index()
in2013.columns = ['hr', 'dow', 'plaza', 'carsin']
out2013 = dfMTA.loc[(dfMTA.loc[:, 'mtadt'].dt.year == 2013) &
                   (dfMTA.loc[:, 'direction'] == 'O'), :].\
                    assign(carsout = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
                        groupby(['hr', dfMTA.loc[:, 'mtadt'].dt.day_of_week, 'plaza']).agg({'carsout': ['sum']}).reset_index()
out2013.columns = ['hr', 'dow', 'plaza', 'carsout']
right = pd.merge(in2013, out2013, on = ['hr', 'dow', 'plaza'], how = 'inner')
right.loc[:, 'ratio2013'] = right.loc[:, 'carsin']/right.loc[:, 'carsout']
right.drop(columns = ['carsin', 'carsout'], inplace=True)
pd.merge(left, right, on = ['hr', 'dow', 'plaza'], how = 'inner')

,hr,dow,plaza,ratio2014,ratio2013
0,0,0,1,1.156617,1.156617
1,0,0,2,1.485974,1.485974
2,0,0,3,1.080547,1.080547
3,0,0,4,0.516120,0.516120
4,0,0,5,1.846699,1.846699
...,...,...,...,...,...
1507,23,6,5,1.267255,1.267255
1508,23,6,6,0.697925,0.697925
1509,23,6,7,1.452002,1.452002
1510,23,6,8,0.568225,0.568225


18. For the day with the most traffic (inbound, outbound and both payment types combined), calculatethe ratio of inbound to outbound traffic over the entire dataset (not by plaza), for each hour. Return three columns, the day-of-the-week of that date, hour and the percent for that hour.

In [72]:
maxday = dfMTA.assign(traffic = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).groupby('mtadt').agg({'traffic': ['max']}).nlargest(1, ('traffic', 'max')).reset_index().loc[:, 'mtadt'].values[0]
maxdow = dfMTA.assign(traffic = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
  groupby('mtadt').agg({'traffic': ['max']}).\
    nlargest(1, ('traffic', 'max')).\
      reset_index().loc[:, 'mtadt'].dt.strftime('%A')
maxdow = dfMTA.loc[(dfMTA.loc[:, 'mtadt'] == maxday), 'mtadt'].dt.strftime('%A').values[0]
df_in = dfMTA.loc[(dfMTA.loc[:, 'mtadt'] == maxday) &
                  (dfMTA.loc[:, 'direction'] == 'I'), :].\
                    assign(traffic = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
                        groupby('hr', as_index = False).agg({'traffic': ['sum']})
df_in.columns = ['hr', 'carsin']
df_out = dfMTA.loc[(dfMTA.loc[:, 'mtadt'] == maxday) &
                  (dfMTA.loc[:, 'direction'] == 'O'), :].\
                    assign(traffic = dfMTA.loc[:, 'vehiclesez'] + dfMTA.loc[:, 'vehiclescash']).\
                        groupby('hr', as_index = False).agg({'traffic': ['sum']})
df_out.columns = ['hr', 'carsout']
temp = pd.merge(df_in, df_out, on = 'hr', how = 'inner')
temp = temp.assign(ratio = temp.loc[:, 'carsin']/temp.loc[:, 'carsout']).drop(columns = ['carsin', 'carsout'])
temp.loc[:, 'dow'] = maxdow
temp

,hr,ratio,dow
0,0,1.528837,Monday
1,1,1.306878,Monday
2,2,1.131972,Monday
3,3,1.080143,Monday
4,4,1.031233,Monday
5,5,1.316895,Monday
6,6,1.621171,Monday
7,7,1.862302,Monday
8,8,1.916593,Monday
9,9,1.807076,Monday
